In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

from joblib import dump

import sys 
import os
import json
sys.path.append("../src")
from data_preparation import DataPreparation

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_12780/176984250.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../data/train.csv")
/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_12780/176984250.py:2: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv("../data/test.csv")


# Préparation des données

In [3]:
preprocess = DataPreparation(train, test)
train, test = preprocess.prepare_data()

Valeurs manquantes du train supprimées ✅
Valeurs manquantes du test supprimées ✅
Variables renommées ✅
Valeurs manquantes numériques imputées ✅
Valeurs manquantes catégorielles imputées ✅


In [9]:
var_explicatives = ["Fm", 'ec_(cm3)', 'ep_(KW)', "Fuel_consumption_", 'Electric_range_(km)', "m_(kg)"]

In [10]:
X = train[var_explicatives]
y = train["Ewltp_(g/km)"]

In [15]:
X = pd.get_dummies(X, dtype='int')

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, test_size=0.25)

In [4]:
train.to_csv("../data/train_clean.csv")
test.to_csv("../data/test_clean.csv")

# Modélisation XGBoost

In [20]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_val, label=y_val)

In [41]:
params = {
    'max_depth': 20,
    'eta': 1,
    'objective': 'reg:squarederror',  
    'eval_metric': 'rmse'  # Utilisation de l'erreur quadratique moyenne comme métrique d'évaluation
}

In [42]:
num_rounds = 200
bst = xgb.train(params, dtrain, num_boost_round=num_rounds)

In [43]:
preds = bst.predict(dtest)

In [44]:
mean_absolute_error(y_val, preds)

3.597934271518895

In [46]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror')

# Définir la grille de paramètres pour la recherche sur grille
param_grid = {
    'max_depth': [5, 10, 15],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'colsample_bytree': [0.3, 0.7]
}

# Initialiser GridSearchCV
grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, 
                           scoring='neg_mean_absolute_error', cv=3, verbose=1)

# Effectuer la recherche sur grille
grid_search.fit(X_train, y_train)

# Résultats de la recherche sur grille
print(f"Meilleurs paramètres: {grid_search.best_params_}")
print(f"Meilleur score MAE: {-grid_search.best_score_}")

# Meilleur modèle entraîné sur l'ensemble des données d'entraînement
best_model = grid_search.best_estimator_

Fitting 3 folds for each of 54 candidates, totalling 162 fits


KeyboardInterrupt: 